In [1]:
!pip install gensim
!pip install nltk

In [55]:
import re  
import pandas as pd 
from time import time  
from collections import defaultdict 
import string 
import multiprocessing
import os
import gensim
import sklearn
from sklearn import linear_model
from collections import Counter
import numpy as np
import scipy
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, cohen_kappa_score, classification_report
from nltk.tokenize import word_tokenize
import pickle

# word2vec
from gensim.models import Word2Vec, KeyedVectors, FastText
from gensim.models.phrases import Phrases, Phraser
from sklearn.model_selection import train_test_split
import logging

import nltk
from nltk.stem import PorterStemmer
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [46]:
path =  "../../Data/train/df_us_train.pickle"
df_us_train = pickle.load(open(path, "rb"))

path =  "../../Data/train/df_es_train.pickle"
df_es_train = pickle.load(open(path, "rb"))

In [49]:
from collections import Counter

punctuation = string.punctuation + "«»“”‘’…—"

stopwords_spanish = pd.read_csv(
    'https://raw.githubusercontent.com/Alir3z4/stop-words/master/spanish.txt'
).values
stopwords_spanish = Counter(stopwords_spanish.flatten().tolist())

stopwords_english = pd.read_csv(
    'https://raw.githubusercontent.com/Alir3z4/stop-words/master/english.txt'
).values
stopwords_english = Counter(stopwords_english.flatten().tolist())

## Contructor de sentence-embeddings

In [70]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

### BOW

In [56]:
class StemmerTokenizer:
    def __init__(self, stopwords):
        self.ps = PorterStemmer()
        self.sw = stopwords
        
    def __call__(self, doc):
        doc_tok = word_tokenize(doc)
        doc_tok = [t for t in doc_tok if t not in self.sw]
        return [self.ps.stem(t) for t in doc_tok]

# Inicializamos tokenizador
tokenizador_english = StemmerTokenizer(stopwords_english)
tokenizador_spanish = StemmerTokenizer(stopwords_spanish)

In [60]:
bow_english = CountVectorizer(
    tokenizer= tokenizador_english,
    ngram_range=(1,1),
)

bow_spanish= CountVectorizer(
    tokenizer= tokenizador_spanish,
    ngram_range=(1,1),
)

ct_bow_english = ColumnTransformer([
    ("BOW", bow_english, "text")
])

ct_bow_spanish = ColumnTransformer([
    ("BOW", bow_spanish, "text")
])

In [61]:
pipe_bow_english = Pipeline(
    steps=[
        ('preprocessing', ct_bow_english)
    ]
)

pipe_bow_spanish = Pipeline(
    steps=[
        ('preprocessing', ct_bow_spanish)
    ]
)